# Chadgpt, llama2 7b version

## Update  12/16/2023
1. Execute the code cell below.
2. After running the cell, restart your session.
- **Reason:** Currently, a session restart is required for the model download. This might be a bug and could be patched in future releases.


In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git
!pip install transformers
!pip install -U accelerate
!pip install accelerate
!pip install bitsandbytes # Instal bits and bytes for inference of the model

In [ ]:
# You need a huggingface token that can access llama2
!huggingface-cli login

# Download the model

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "danjie/Chadgpt-Llama2-7b"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

# Inference

In [ ]:
def talk_with_llm(tweet: str) -> str:
    # Encode and move tensor into cuda if applicable.
    encoded_input = tokenizer(tweet, return_tensors='pt')
    encoded_input = {k: v.to("cuda") for k, v in encoded_input.items()}

    output = model.generate(**encoded_input, max_new_tokens=64)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [ ]:
talk_with_llm("<User> Hey, can you help me pick up my kids today? I'll have to run to a dentist appointment. \n<Assistant>")